In [1]:
from file_extraction import *
from to_dataframe import *
from clean_dfs import *
from 
import pandas as pd

In [2]:
files_dict, academy_csvs_file_names = extract()
dataframes = convert(files_dict, academy_csvs_file_names)

In [3]:
df_academy_csv = dataframes['academy_csv']
df_academy_csv = clean_academy_csv(df_academy_csv)
display(df_academy_csv)

,name,trainer,course,date,Analytic_W1,Independent_W1,Determined_W1,Professional_W1,Studious_W1,Imaginative_W1,...,Determined_W9,Professional_W9,Studious_W9,Imaginative_W9,Analytic_W10,Independent_W10,Determined_W10,Professional_W10,Studious_W10,Imaginative_W10
0,QUINTUS PENELLA,Gregor Gomez,Business 20,2019-02-11,1,2,2,1,2,2,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,SIMON MURREY,Gregor Gomez,Business 20,2019-02-11,6,1,1,2,4,2,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,GUSTAF LUDE,Gregor Gomez,Business 20,2019-02-11,6,4,1,1,2,3,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,YOLANDA FOSSE,Gregor Gomez,Business 20,2019-02-11,2,1,2,3,3,3,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,LYNNETT SWIN,Gregor Gomez,Business 20,2019-02-11,2,2,4,5,1,2,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,MORITZ MOSEDALL,Mohammad Velazquez,Engineering 29,2019-12-30,1,1,5,1,2,6,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,CHAIM INSEAL,Mohammad Velazquez,Engineering 29,2019-12-30,1,3,3,4,1,2,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
6,GERTRUDA SYDDIE,Mohammad Velazquez,Engineering 29,2019-12-30,3,1,2,8,1,4,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
7,THOM DERWIN,Mohammad Velazquez,Engineering 29,2019-12-30,3,7,3,3,3,1,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [4]:
df_csv = dataframes['csv']
df_csv = clean_talent_csv(df_csv)
display(df_csv)

,id,name,gender,dob,email,city,address,postcode,phone_number,uni,degree,invited_by,invite_date
0,1,ESME TRUSSLOVE,F,1994-08-04,etrusslove0@google.es,Swindon,22056 Lerdahl Avenue,SN1,442957830228,"Saint George's Hospital Medical School, Univer...",2.1,BRUNO BELLBROOK,2019-04-10
1,2,MATTHAEUS AUDAS,M,NaT,maudas1@mapquest.com,Charlton,263 Nelson Trail,OX12,449577280155,Keele University,2.1,DORIS BELLASIS,2019-04-30
2,3,CHEREY TOLLFREE,F,1992-12-08,ctollfree2@netvibes.com,Weston,69 Coleman Court,GU32,445887496002,"King's College London, University of London",2.1,GISMO TILLING,2019-04-25
3,4,ERYN SPEERS,F,NaT,espeers3@shinystat.com,Sutton,0166 Daystar Drive,CT15,441487870613,University of Edinburgh,2.1,NaN,NaT
4,5,THEADORA BERKELAY,F,1995-11-03,tberkelay4@godaddy.com,Upton,6 Mandrake Crossing,WF9,448414683619,University of Leicester,2.1,STACEY BROAD,2019-04-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,416,CLYVE GILLHESPY,M,1995-12-15,cgillhespybj@buzzfeed.com,Birmingham,33 Almo Avenue,B40,449043432218,University of Liverpool,2.1,BRUNO BELLBROOK,2019-09-26
416,417,VACLAV PIETESCH,M,1994-11-09,vpieteschbk@mac.com,Whitwell,6476 Hoffman Terrace,DL10,444556316125,Sheffield Hallam University,2.2,STACEY BROAD,2019-09-12
417,418,KASSI LUCIO,F,1994-04-24,kluciobl@exblog.jp,Normanton,06 Fulton Center,LE15,448343429323,University of Buckingham,2.1,FIFI ETON,2019-09-03
418,419,VIVIANNA LETTY,F,NaT,vlettybm@google.com.hk,Sheffield,78314 Fisk Plaza,S1,445347583140,Leeds Metropolitan University,1.0,BRUNO BELLBROOK,2019-09-19


In [5]:
df_json = dataframes['json']
df_json = clean_talent_json(df_json)
display(df_json)

,name,date,tech_self_score,strengths,weaknesses,self_development,geo_flex,financial_support_self,result,course_interest
0,STILLMANN CASTANO,2019-08-22,"{'C#': 6, 'JAVA': 5, 'R': 2, 'JAVASCRIPT': 2}",[Charisma],"[Distracted, Impulsive, Introverted]",YES,YES,YES,PASS,BUSINESS
0,HILARY WILLMORE,2019-08-01,"{'PYTHON': 1, 'C#': 4, 'JAVA': 2, 'C++': 4}","[Patient, Curious, Problem Solving]","[Overbearing, Chatty, Indifferent]",NO,YES,YES,FAIL,DATA
0,EFREM WHIPPLE,2019-08-22,"{'RUBY': 4, 'C++': 4}","[Courteous, Independent, Patient]","[Introverted, Impulsive, Anxious]",YES,YES,YES,PASS,BUSINESS
0,SYDEL FENNE,2019-08-28,"{'JAVA': 3, 'SPSS': 4}",[Passionate],"[Perfectionist, Sensitive]",YES,YES,YES,PASS,DATA
0,MICHEL LEBARREE,2019-08-07,"{'PYTHON': 3, 'JAVA': 4, 'RUBY': 1, 'R': 2, 'P...",[Versatile],"[Controlling, Perfectionist, Chatty]",YES,YES,YES,PASS,ENGINEERING
...,...,...,...,...,...,...,...,...,...,...
0,JACKY REILINGEN,2019-04-04,"{'C#': 2, 'JAVA': 6, 'R': 1, 'SPSS': 4}",[Versatile],"[Indifferent, Intolerant, Introverted]",YES,NO,YES,FAIL,BUSINESS
0,PHILLIS LYFIELD,2019-04-10,"{'C#': 4, 'JAVA': 4, 'RUBY': 4, 'PHP': 1}","[Organisation, Independent, Determined]","[Sensitive, Overbearing, Impatient]",YES,YES,YES,PASS,ENGINEERING
0,CELLE BARLAS,2019-04-16,"{'R': 2, 'C++': 1, 'JAVASCRIPT': 4}",[Problem Solving],[Critical],YES,YES,YES,PASS,ENGINEERING
0,SCOTT DUNY,2019-04-11,{'RUBY': 3},"[Reliable, Perfectionism, Problem Solving]",[Stubborn],YES,YES,YES,PASS,DATA


In [6]:
df_txt = dataframes['txt']
df_txt = clean_talent_txt(df_txt)
display(df_txt)

,name,psychometrics_score,presentation_score,academy,date
0,HILARY WILLMORE,51,19,Birmingham Academy,2019-08-01
1,ORLY LORENS,51,19,Birmingham Academy,2019-08-01
2,ALVIE BLEACKLY,55,16,Birmingham Academy,2019-08-01
3,DECK ITZCHAKI,59,21,Birmingham Academy,2019-08-01
4,WILT PENRITT,66,25,Birmingham Academy,2019-08-01
...,...,...,...,...,...
15,GODIVA ANDREW,46,19,London Academy,2019-10-09
16,DORALIA GAPPER,55,22,London Academy,2019-10-09
17,DOE EISOLD,57,20,London Academy,2019-10-09
18,JUDY FINDERS,56,22,London Academy,2019-10-09


# Generate Candidates Table

In [54]:
def get_person_ids(applicants):
    applicants = applicants.reset_index().reset_index().drop(['index', 'id'], axis=1).rename(columns={'level_0':'person_id'})
    return applicants

 

def generate_candidates_df(applicants, talents, sparta_day_results):
    applicants_and_talents = applicants.merge(talents, 
                                              left_on=['name', 'invite_date'], 
                                              right_on=['name', 'date'], 
                                              how='left')

    candidates = applicants_and_talents.merge(sparta_day_results, 
                                              left_on=['name', 'invite_date'], 
                                              right_on=['name', 'date'], 
                                              how='left')

    candidates = candidates.drop(candidates.index[candidates.person_id.duplicated().tolist()].tolist())
    candidates.reset_index(drop=True, inplace=True)
    candidates['person_id'] = candidates.index + 1
    return candidates

In [55]:
# dont use this candidates table
candidates = generate_candidates_df(get_person_ids(df_csv), df_json, df_txt)
display(candidates)

,person_id,name,gender,dob,email,city,address,postcode,phone_number,uni,...,weaknesses,self_development,geo_flex,financial_support_self,result,course_interest,psychometrics_score,presentation_score,academy,date_y
0,1,ESME TRUSSLOVE,F,1994-08-04,etrusslove0@google.es,Swindon,22056 Lerdahl Avenue,SN1,442957830228,"Saint George's Hospital Medical School, Univer...",...,NaN,NaN,NaN,NaN,NaN,NaN,39.0,23.0,London Academy,2019-04-10
1,2,MATTHAEUS AUDAS,M,NaT,maudas1@mapquest.com,Charlton,263 Nelson Trail,OX12,449577280155,Keele University,...,"[Critical, Introverted, Immature]",YES,YES,YES,PASS,DATA,59.0,20.0,London Academy,2019-04-30
2,3,CHEREY TOLLFREE,F,1992-12-08,ctollfree2@netvibes.com,Weston,69 Coleman Court,GU32,445887496002,"King's College London, University of London",...,[Undisciplined],YES,YES,YES,PASS,BUSINESS,59.0,18.0,Birmingham Academy,2019-04-25
3,4,ERYN SPEERS,F,NaT,espeers3@shinystat.com,Sutton,0166 Daystar Drive,CT15,441487870613,University of Edinburgh,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,5,THEADORA BERKELAY,F,1995-11-03,tberkelay4@godaddy.com,Upton,6 Mandrake Crossing,WF9,448414683619,University of Leicester,...,NaN,NaN,NaN,NaN,NaN,NaN,59.0,12.0,London Academy,2019-04-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4686,4687,CLYVE GILLHESPY,M,1995-12-15,cgillhespybj@buzzfeed.com,Birmingham,33 Almo Avenue,B40,449043432218,University of Liverpool,...,"[Indecisive, Introverted]",YES,YES,YES,PASS,BUSINESS,59.0,24.0,Birmingham Academy,2019-09-26
4687,4688,VACLAV PIETESCH,M,1994-11-09,vpieteschbk@mac.com,Whitwell,6476 Hoffman Terrace,DL10,444556316125,Sheffield Hallam University,...,"[Passive, Selfish, Chaotic]",YES,YES,NO,FAIL,ENGINEERING,61.0,27.0,Birmingham Academy,2019-09-12
4688,4689,KASSI LUCIO,F,1994-04-24,kluciobl@exblog.jp,Normanton,06 Fulton Center,LE15,448343429323,University of Buckingham,...,NaN,NaN,NaN,NaN,NaN,NaN,60.0,24.0,London Academy,2019-09-03
4689,4690,VIVIANNA LETTY,F,NaT,vlettybm@google.com.hk,Sheffield,78314 Fisk Plaza,S1,445347583140,Leeds Metropolitan University,...,NaN,NaN,NaN,NaN,NaN,NaN,56.0,19.0,Birmingham Academy,2019-09-19


# Generate Academy Table

In [9]:
# makes an address table dataframe 
def generate_address_df(df):
    addresses = list()
    # for each row in the talent csv files, add each unique address, city and postcode to a list
    for index, row in df.iterrows():
                address_list = list()
                address_list.append(row["address"])
                address_list.append(row["city"])
                address_list.append(row["postcode"])
                if address_list not in addresses:
                    addresses.append(address_list)
                    
    address_table = pd.DataFrame(addresses, columns=["address", "city", "postcode"])
    address_table.index = address_table.index + 1
    address_table['address_id'] = address_table.index
    return address_table

In [10]:
address_table = generate_address_df(df_csv)
display(address_table)

,address,city,postcode,address_id
1,22056 Lerdahl Avenue,Swindon,SN1,1
2,263 Nelson Trail,Charlton,OX12,2
3,69 Coleman Court,Weston,GU32,3
4,0166 Daystar Drive,Sutton,CT15,4
5,6 Mandrake Crossing,Upton,WF9,5
...,...,...,...,...
4594,33 Almo Avenue,Birmingham,B40,4594
4595,6476 Hoffman Terrace,Whitwell,DL10,4595
4596,06 Fulton Center,Normanton,LE15,4596
4597,78314 Fisk Plaza,Sheffield,S1,4597


In [56]:
def address_id_in_candidates(candidates, address_table):
    merged = pd.merge(candidates, address_table, on='address')
    merged = merged.drop(columns=['city_x', 'address', 'postcode_x', 'city_y', 'postcode_y', 'date_y', 'weaknesses', 'strengths'])
    merged = merged.rename(columns={"date_x": "date"})
    return merged

In [57]:
# use this candidates table
candidates = address_id_in_candidates(candidates, address_table)
display(candidates)

,person_id,name,gender,dob,email,phone_number,uni,degree,invited_by,invite_date,...,tech_self_score,self_development,geo_flex,financial_support_self,result,course_interest,psychometrics_score,presentation_score,academy,address_id
0,1,ESME TRUSSLOVE,F,1994-08-04,etrusslove0@google.es,442957830228,"Saint George's Hospital Medical School, Univer...",2.1,BRUNO BELLBROOK,2019-04-10,...,NaN,NaN,NaN,NaN,NaN,NaN,39.0,23.0,London Academy,1
1,2,MATTHAEUS AUDAS,M,NaT,maudas1@mapquest.com,449577280155,Keele University,2.1,DORIS BELLASIS,2019-04-30,...,"{'PYTHON': 3, 'JAVA': 3, 'SPSS': 4, 'C++': 2}",YES,YES,YES,PASS,DATA,59.0,20.0,London Academy,2
2,3,CHEREY TOLLFREE,F,1992-12-08,ctollfree2@netvibes.com,445887496002,"King's College London, University of London",2.1,GISMO TILLING,2019-04-25,...,"{'C#': 1, 'JAVA': 3, 'RUBY': 3}",YES,YES,YES,PASS,BUSINESS,59.0,18.0,Birmingham Academy,3
3,4,ERYN SPEERS,F,NaT,espeers3@shinystat.com,441487870613,University of Edinburgh,2.1,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
4,5,THEADORA BERKELAY,F,1995-11-03,tberkelay4@godaddy.com,448414683619,University of Leicester,2.1,STACEY BROAD,2019-04-02,...,NaN,NaN,NaN,NaN,NaN,NaN,59.0,12.0,London Academy,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4698,4687,CLYVE GILLHESPY,M,1995-12-15,cgillhespybj@buzzfeed.com,449043432218,University of Liverpool,2.1,BRUNO BELLBROOK,2019-09-26,...,"{'C#': 4, 'JAVA': 4, 'R': 3}",YES,YES,YES,PASS,BUSINESS,59.0,24.0,Birmingham Academy,4594
4699,4688,VACLAV PIETESCH,M,1994-11-09,vpieteschbk@mac.com,444556316125,Sheffield Hallam University,2.2,STACEY BROAD,2019-09-12,...,"{'PYTHON': 1, 'JAVA': 2, 'PHP': 3, 'C++': 4}",YES,YES,NO,FAIL,ENGINEERING,61.0,27.0,Birmingham Academy,4595
4700,4689,KASSI LUCIO,F,1994-04-24,kluciobl@exblog.jp,448343429323,University of Buckingham,2.1,FIFI ETON,2019-09-03,...,NaN,NaN,NaN,NaN,NaN,NaN,60.0,24.0,London Academy,4596
4701,4690,VIVIANNA LETTY,F,NaT,vlettybm@google.com.hk,445347583140,Leeds Metropolitan University,1.0,BRUNO BELLBROOK,2019-09-19,...,NaN,NaN,NaN,NaN,NaN,NaN,56.0,19.0,Birmingham Academy,4597


# Generate Courses Table

In [13]:
def generate_courses_table(dataframe):
    courses_df  = dataframe
    courses_df = pd.DataFrame().assign(course = courses_df['course'], date = courses_df['date'])
    courses_df['course_id'] = courses_df.groupby(['course']).ngroup()
    courses_df = courses_df[['course', 'course_id', 'date']].copy()
    courses_df.set_index('course_id', inplace=True)
    courses_df.groupby(['course','date'])
    courses_df = courses_df.drop_duplicates()
    courses_df = courses_df.reset_index()
    courses_df.index = courses_df.index + 1
    courses_df['course_id'] = courses_df.index
    courses_df = courses_df.rename(columns={"course": "course_name"})
    
    return courses_df

In [14]:
courses_table = generate_courses_table(df_academy_csv)
display(courses_table)

,course_id,course_name,date
1,1,Business 20,2019-02-11
2,2,Business 21,2019-03-18
3,3,Business 22,2019-04-15
4,4,Business 23,2019-05-20
5,5,Business 24,2019-07-15
6,6,Business 25,2019-07-29
7,7,Business 26,2019-08-12
8,8,Business 27,2019-09-16
9,9,Business 28,2019-10-21
10,10,Business 29,2019-11-18


# Generate Weakness Table

In [15]:
# makes a weakness table dataframe
def generate_weakness_df(df):
    weaknesses_list = list()
    
    # for each row in the dataframe add distinct weaknesses to a list
    for index, row in df.iterrows():
        for weakness in row["weaknesses"]:
            if weakness not in weaknesses_list:
                weaknesses_list.append(weakness)
                
    # turn weakneseses list into a dataframe and add a column for the weakness id            
    weakness_table = pd.DataFrame(weaknesses_list, columns=['weakness'])
    weakness_table.index = weakness_table.index + 1
    weakness_table['weakness_id'] = weakness_table.index
    return weakness_table          

In [16]:
weakness_df = generate_weakness_df(df_json)
display(weakness_df)

,weakness,weakness_id
1,Distracted,1
2,Impulsive,2
3,Introverted,3
4,Overbearing,4
5,Chatty,5
6,Indifferent,6
7,Anxious,7
8,Perfectionist,8
9,Sensitive,9
10,Controlling,10


In [17]:
# takes talent jsons, talent csv, weakness dataframe as arguments
def generate_weakness_junc_df(candidates, weakness_df):
    weakness_junc_list = list()
    
    # for each row in the merged dataframe add (id, weakness) to a new dataframe for each weakness in weaknesses list
    for index, row in candidates.iterrows():
        person_id = row['person_id']
        weaknesses = row['weaknesses']
        
        if type(weaknesses) == list:
            for weakness in weaknesses:
                weakness_junc_list.append([person_id, weakness])
    
    weakness_junc = pd.DataFrame(weakness_junc_list, columns=['person_id', 'weakness'])
    
    weakness_junc = pd.merge(weakness_junc, weakness_df, on='weakness')[['person_id', 'weakness_id']] 
        
    return weakness_junc      

In [18]:
weakness_junc_df = generate_weakness_junc_df(candidates, weakness_df)
display(weakness_junc_df)

,person_id,weakness_id
0,2,21
1,2236,21
2,2895,21
3,3937,21
4,35,21
...,...,...
5513,4634,2
5514,4653,2
5515,4658,2
5516,4666,2


# Generate Strengths Table

In [19]:
def generate_strengths_df(df):
    # Create a new DataFrame for strengths column only
    column_names = ['name', 'strengths']
    strengths = df[column_names]

    # Initialize an empty list to store row data
    strengths_list = []

    # Iterate over all rows
    for index, row in strengths.iterrows():
        strengths_list_row = row['strengths']

        # Iterate over each strength in the list
        for strength in strengths_list_row:
            if strength not in strengths_list:
                # Append row data to the list
                strengths_list.append(strength)

    # Create a new DataFrame from the list
    strengths_df = pd.DataFrame({'strength': strengths_list})
    strengths_df.index = strengths_df.index + 1
    strengths_df['strength_id'] = strengths_df.index
    # Return the new DataFrame
    return strengths_df

In [20]:
strengths_df = generate_strengths_df(df_json)
display(strengths_df)

,strength,strength_id
1,Charisma,1
2,Patient,2
3,Curious,3
4,Problem Solving,4
5,Courteous,5
6,Independent,6
7,Passionate,7
8,Versatile,8
9,Rational,9
10,Collaboration,10


In [21]:
# takes talent jsons, talent csv, weakness dataframe as arguments
def generate_strengths_junc_df(candidates, strengths_df):
    strengths_junc_list = list()
    
    for index, row in candidates.iterrows():
        person_id = row['person_id']
        strengths = row['strengths']
        
        if type(strengths) == list:
            for strength in strengths:
                strengths_junc_list.append([person_id, strength])
    
    strengths_junc = pd.DataFrame(strengths_junc_list, columns=['person_id', 'strength'])

    strengths_junc = pd.merge(strengths_junc, strengths_df, on='strength')[['person_id', 'strength_id']] 
        
    return strengths_junc

In [22]:
strengths_junc_df = generate_strengths_junc_df(candidates, strengths_df)
display(strengths_junc_df)

,person_id,strength_id
0,2,12
1,76,12
2,377,12
3,531,12
4,1729,12
...,...,...
5604,4601,25
5605,4611,25
5606,4616,25
5607,4639,25


# Generate Tech Score Table

In [27]:
def generate_tss_df(df, candidates):
    #Create names lists
    names = []
    technologies = []
    self_scores = []

    #Loop through dataframe
    for index, row in df.iterrows():
        try:
            techs = list(row['tech_self_score'].keys())
            for tech in techs:
                names.append(row['name'].upper())
                technologies.append(tech)
                self_scores.append(row['tech_self_score'][tech])
        except AttributeError:
            continue

    tech_self_scores = pd.DataFrame({'name':names, 'technology':technologies, 'self_score': self_scores})
    merged = pd.merge(tech_self_scores, candidates, on='name')[['person_id', 'technology', 'self_score']] 
    
    return merged

In [28]:
tech_scores = generate_tss_df(df_json, candidates)
display(tech_scores)

,person_id,technology,self_score
0,380,C#,6
1,380,JAVA,5
2,380,R,2
3,380,JAVASCRIPT,2
4,381,PYTHON,1
...,...,...,...
9798,377,RUBY,3
9799,379,PYTHON,2
9800,379,C#,3
9801,379,JAVA,3


# Generate Academy Table

In [31]:
def generate_academy_table(df_academy_csv, courses_table, candidates):
    merged = pd.merge(df_academy_csv, courses_table, left_on='course', right_on='course_name')[['name', 'course_id']]
    academy_table = pd.merge(merged, candidates, on='name')[['person_id', 'course_id']] 
    return academy_table

In [32]:
academy_table = generate_academy_table(df_academy_csv, courses_table, candidates)
display(academy_table)

,person_id,course_id
0,1769,1
1,1696,1
2,1663,1
3,1573,1
4,1661,1
...,...,...
393,844,36
394,1091,36
395,847,36
396,949,36


# Generate Trainers Table

In [50]:
def generate_trainers_table(df):
    old_name = "Ely Kely"
    new_name = "Elly Kelly"
    df.loc[:, ["trainer"]] = df.loc[:, ["trainer"]].replace(old_name, new_name)
    trainers_course = df[["trainer", "course", "date"]]
    trainer_course = trainers_course.drop_duplicates(subset = ["date"], keep= "first")
    trainer_course['trainer_id'] = range(1, len(trainer_course) + 1)
    trainers = trainer_course.set_index('trainer_id')
    trainers_table = trainers.drop(['course', 'date'], axis=1).drop_duplicates()
    trainers = trainers.rename(columns={"course": "course_name"})
    trainers['trainer_id'] = trainers.index
    return trainers, trainers_table


In [51]:
trainers, trainers_table = generate_trainers_table(df_academy_csv)
display(trainers_table)

/var/folders/9x/277hw0sj1jzdvbfm4bqgz_540000gn/T/ipykernel_21246/140333513.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainer_course['trainer_id'] = range(1, len(trainer_course) + 1)


,trainer
trainer_id,
1,Gregor Gomez
2,Bruce Lugo
4,Neil Mccarthy
5,Rachel Richard
7,Hamzah Melia
8,Burhan Milner
9,Elly Kelly
12,Trixie Orange
13,John Sandbox


In [35]:
display(trainers)

,trainer,course_name,date,trainer_id
trainer_id,,,,
1,Gregor Gomez,Business 20,2019-02-11,1
2,Bruce Lugo,Business 21,2019-03-18,2
3,Gregor Gomez,Business 22,2019-04-15,3
4,Neil Mccarthy,Business 23,2019-05-20,4
5,Rachel Richard,Business 24,2019-07-15,5
6,Gregor Gomez,Business 25,2019-07-29,6
7,Hamzah Melia,Business 26,2019-08-12,7
8,Burhan Milner,Business 27,2019-09-16,8
9,Elly Kelly,Business 28,2019-10-21,9


In [36]:
def generate_trainers_junc(df_trainers, df_courses):
    trainers_junc = pd.merge(df_trainers, df_courses, on='course_name')[['course_id', 'trainer_id']] 
    return trainers_junc

In [37]:
trainers_junc = generate_trainers_junc(trainers, courses_table)
display(trainers_junc)

,course_id,trainer_id
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5
5,6,6
6,7,7
7,8,8
8,9,9
9,10,10


# Generate Academy Results Table

In [40]:
def use(key: str):
    s3 = boto3.client('s3')
    obj = s3.get_object(Bucket='data-eng-228-final-project', Key=key)
    filetype = key[key.index('.'):]
    if filetype == '.csv':
        data = pd.read_csv(obj['Body'])
    elif filetype == '.json':
        data = json.load(obj['Body'])
    elif filetype == '.txt':
        data = obj['Body'].read().decode('utf-8')
    return data


def make_wide(table):
    academy_results_wide = table.pivot(index=['name', 'week_no'],
                                       columns='attribute',
                                       values='score')
    academy_results = academy_results_wide.reset_index()
    return academy_results


def make_upper(df, attribute: str='name'):
    return df[attribute].transform(lambda attr: attr.upper())


def transform_acares(keystrings: list):
    tables = []

    for keystring in keystrings:
        current_ar = use(keystring)
        current_ar['name'] = make_upper(current_ar, 'name')
        raw_attributes = current_ar.columns[2:]

        names = []
        week_numbers = []
        attributes = []
        scores = []

        for name in current_ar['name']:
            for attribute in raw_attributes:
                names.append(name)
                week_numbers.append(attribute[attribute.index('W') + 1:])
                attributes.append(attribute[:attribute.index('_')])
                scores.append(float(current_ar[current_ar['name'] == name][attribute].tolist()[0]))
        academy_results_long = pd.DataFrame({'name': names,
                                             'week_no': week_numbers,
                                             'attribute': attributes,
                                             'score': scores})
        tables.append(academy_results_long)
    return pd.concat(tables)

In [41]:
s3_client = boto3.client('s3')
files_dict = {'academy_csv': [], 'json': [], 'txt': [], 'csv': []}
academy_csvs_file_names = extract_file_type(s3_client, 'Academy', files_dict, 'csv')
df1 = transform_acares(academy_csvs_file_names)
academy_results = make_wide(df1)

In [42]:
def join_person_id_to_acares(candidates, academy_results):
    passed_candidates = candidates[(candidates.result == 'PASS').tolist()]
    passed_candidates

 

    academy_results = academy_results.merge(passed_candidates[['person_id', 'name']], left_on='name', right_on='name')
    return academy_results

In [44]:
join_person_id_to_acares(candidates, academy_results)

,name,week_no,Analytic,Determined,Imaginative,Independent,Professional,Studious,person_id
0,ADAH SPENCERS,1,5.0,2.0,3.0,3.0,5.0,4.0,288
1,ADAH SPENCERS,2,4.0,1.0,1.0,5.0,1.0,4.0,288
2,ADAH SPENCERS,3,6.0,1.0,2.0,3.0,7.0,2.0,288
3,ADAH SPENCERS,4,4.0,8.0,5.0,6.0,7.0,3.0,288
4,ADAH SPENCERS,5,2.0,8.0,5.0,5.0,3.0,3.0,288
...,...,...,...,...,...,...,...,...,...
3261,ZOLLIE DANKS,4,8.0,3.0,5.0,4.0,6.0,6.0,4579
3262,ZOLLIE DANKS,5,5.0,5.0,8.0,4.0,7.0,7.0,4579
3263,ZOLLIE DANKS,6,7.0,7.0,7.0,8.0,7.0,4.0,4579
3264,ZOLLIE DANKS,7,4.0,8.0,6.0,8.0,8.0,8.0,4579
